# Concat multiple iterations of sampling for coherent layout in Gephi

In [1]:
import numpy as np
import pandas as pd
import networkx as nx

from fairpair import *

In [2]:
G = FairPairGraph()
G.generate_groups(15, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.3, scale=0.2) # give a disadvantage to the minority

In [3]:
H = nx.MultiDiGraph()
H.add_nodes_from(G.nodes(data=True))

In [4]:
for iteration in range(10):
    sampling = ProbKnockoutSampling(G, warn=False)
    sampling.apply(iter=1, k=1)
    # get the current graph and add it to the multigraph, but label edges by iteration
    edges = [(i, j, {**data, 'iteration': iteration}) for i, j, data in G.edges(data=True)]
    H.add_edges_from(edges)
    print(len(H.edges))

8
20
32
44
58
72
86
100
116
134


In [5]:
list(H.edges(data=True))[:3]

[(2, 5, {'wins': 0, 'weight': 0.0, 'iteration': 0}),
 (2, 5, {'wins': 0, 'weight': 0.0, 'iteration': 1}),
 (2, 5, {'wins': 0, 'weight': 0.0, 'iteration': 2})]

In [6]:
for node in H.nodes(data=True):
    node[1]['minority'] = bool(node[1]['minority']) # convert from numpy
nx.write_gexf(H, 'data/multigraphtest.gexf')